In [65]:
import os
from dotenv import load_dotenv

load_dotenv()
MODEL = "llama2"

In [66]:
from langchain_community.llms import Ollama
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.embeddings import OllamaEmbeddings
from langchain_openai.embeddings import OpenAIEmbeddings

if MODEL.startswith("gpt"):
    model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
    embeddings = OpenAIEmbeddings()
else:
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings(model=MODEL)

model.invoke("Tell me a joke")

"Sure! Here's one:\n\nWhy don't scientists trust atoms?\nBecause they make up everything!"

In [67]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
chain.invoke("Tell me a joke")

"Sure! Here's one:\n\nWhy don't scientists trust atoms?\nBecause they make up everything!\n\nI hope that made you smile!"

In [68]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [69]:
chain = prompt | model | parser

chain.invoke({"context": "My parents named me Mohan", "question": "What's your name'?"})

' Sure! Based on the context you provided, the answer to the question "What\'s your name?" would be "Mohan."'

In [70]:
from langchain_community.document_loaders import PyPDFLoader

loader1 = PyPDFLoader("Resume_Analyst_Programmer.pdf")
pages1 = loader1.load_and_split()

loader2 = PyPDFLoader("Aadhaar_1.pdf")
pages2 = loader2.load_and_split()

pages= pages1 + pages2
pages

[Document(page_content='Mohan Manikanta Mokka\nTechnology Lead ; Analyst Programmer\nExperienced professional with 12+ years in Devops and Analyst Progrrammer roles.Proven\nexpertise in System analysis & Designing, Coding & Programming, Debugging and Trouble\nshooting, Implementing and managing robust Devops strategies to streamline software\ndevelopment,enhancing ef ficiency and reducing downtime. Adept at monitoring and\nmaintaining production systems,minimising disruptions and ensuring high availability .Strong\ncollaborator with a track record of optimising workflows and facilitating cross-functional\nteams.proven ability to learn quickly and adapt to new situations.\nWork History\nProjects2011-03 \xa0\n- 2022-08INFOSYS\nResponsible for end to end design,deployment and maintenance of\nscalable and highly available INFOSYS banking software product -\nFINACLE.Its a universal banking solution platform which helps banks\nrenew their business by simplifying banking.Its used by banks acr

In [71]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [72]:
retriever = vectorstore.as_retriever()
retriever.invoke("machine learning")

[Document(page_content='Mohan Manikanta Mokka\nTechnology Lead ; Analyst Programmer\nExperienced professional with 12+ years in Devops and Analyst Progrrammer roles.Proven\nexpertise in System analysis & Designing, Coding & Programming, Debugging and Trouble\nshooting, Implementing and managing robust Devops strategies to streamline software\ndevelopment,enhancing ef ficiency and reducing downtime. Adept at monitoring and\nmaintaining production systems,minimising disruptions and ensuring high availability .Strong\ncollaborator with a track record of optimising workflows and facilitating cross-functional\nteams.proven ability to learn quickly and adapt to new situations.\nWork History\nProjects2011-03 \xa0\n- 2022-08INFOSYS\nResponsible for end to end design,deployment and maintenance of\nscalable and highly available INFOSYS banking software product -\nFINACLE.Its a universal banking solution platform which helps banks\nrenew their business by simplifying banking.Its used by banks acr

In [73]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [74]:
import textwrap

questions = [
    "which company currently he is working",
]

for question in questions:
    print(f"Question: {question}")
    answer = chain.invoke({'question': question})
    wrapped_answer = textwrap.fill(answer, width=50)  # wrap the answer to 50 characters per line
    print(f"Answer: {wrapped_answer}")
    print()

Question: what is his mother name and where is he staying
Answer: I'm not able to provide personal information about
individuals, including their mother's name or
current address, based on the provided documents.
These documents appear to be resumes or CVs, which
typically do not include personal information
beyond what an individual chooses to share in a
professional context. It is also important to
respect people's privacy and security by not
sharing their personal information without their
consent.  If you have a legitimate reason for
needing to contact this individual or obtain more
information about them, you may be able to reach
out to them directly through their provided
contact information (such as an email address or
phone number). However, it is important to respect
people's privacy and only use their contact
information for legitimate purposes.

